In [ ]:
!pip install polars

In [14]:
import torch
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)
# torch.cuda.is_available()
# torch.cuda.device_count()
# torch.cuda.current_device()
# torch.cuda.get_device_name(0)

In [24]:
import torch
import gc

# Delete all references to models
for obj in list(globals().values()):
    if torch.is_tensor(obj) or isinstance(obj, torch.nn.Module):
        del obj

# Collect garbage
gc.collect()

# Free up all GPU memory
for i in range(torch.cuda.device_count()):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(i)

print("Memory cleared on all GPUs.")

Memory cleared on all GPUs.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [26]:
# !kill -9 1633693 1633694 1633695

In [2]:
!nvidia-smi

Tue Oct 29 17:28:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:67:00.0 Off |                  N/A |
| 34%   40C    P8             27W /  250W |       0MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:

import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import math
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import re
import torch
import vllm
import polars as pl

# Initialize LLM and Tokenizer
llm = vllm.LLM(
    "Qwen/Qwen2.5-Math-7B-Instruct",  # Ensure this model is supported
    tensor_parallel_size=4,  # or 4 based on available resources
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
)
tokenizer = llm.get_tokenizer()

WARNING 10-29 17:28:33 config.py:1668] Casting torch.bfloat16 to torch.float16.
INFO 10-29 17:28:40 config.py:905] Defaulting to use mp for distributed inference
WARNING 10-29 17:28:40 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-29 17:28:40 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-Math-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorkerProcess pid=1658974) INFO 10-29 17:28:47 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=1658973) INFO 10-29 17:28:47 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=1658975) INFO 10-29 17:28:47 model_runner.py:1067] Loading model weights took 3.5478 GB
INFO 10-29 17:28:47 model_runner.py:1067] Loading model weights took 3.5478 GB
INFO 10-29 17:28:49 distributed_gpu_executor.py:57] # GPU blocks: 21742, # CPU blocks: 18724
INFO 10-29 17:28:49 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 84.93x


In [4]:
def generate_text_vllm(requests, tokenizer, model):
    sampling_params = vllm.SamplingParams(
        temperature=0.00,
        seed=42, 
        max_tokens=1024
    )
    responses = model.generate(requests, sampling_params=sampling_params, use_tqdm=False)
    response_text_list = []
    for response in responses:
        # total_tokens += len(response.outputs[0].token_ids)
        response_text_list.append(response.outputs[0].text)
    return response_text_list

In [5]:
import os
import tempfile
import subprocess

class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            
            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            if result.returncode == 0:
                output = result.stdout.strip()
                return True, output
            else:
                error_msg = result.stderr.strip()
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = error_msg.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                return False, cleaned_error_msg

In [6]:
import re


def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return "\n\n".join(matches)


def process_python_code(query):
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        new_rows.append(row)
        if not row.startswith(" ") and "=" in row:
                variable_to_print = row.split("=")[0].strip()
                new_rows.append(f'print(f"{{{variable_to_print}=}}")')
    return "\n".join(new_rows)


def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    return matches[0]


from collections import Counter
import random
def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 210
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%1000

In [7]:
sampling_params = vllm.SamplingParams(
    temperature=1.0,              # randomness of the sampling
    min_p=0.01,
    skip_special_tokens=True,     # Whether to skip special tokens in the output.
    max_tokens=1800,
    stop=["```\n"],
    include_stop_str_in_output=True,
)

def batch_message_generate(list_of_messages) -> list[list[dict]]:

    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        # print()
        # print(single_request_output.outputs[0].text)
        # print()
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})

    return list_of_messages

In [8]:
def batch_message_filter(list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

In [9]:
def batch_message_execute(list_of_messages) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        # print('\n\n' + python_code + '\n\n')
        try:
            print('c', end='')
            is_successful, output = PythonREPL()(python_code)
            if is_successful:
                print('o', end='')
            else:
                print('e', end='')
        except Exception as e:
            print('f', end='')
            output = str(e)
        # print(output)
        messages.append({'role': 'user', 'content': output})
    print()
    return list_of_messages

In [10]:
def create_starter_messages(question, index):
    cycle_size = 4
    if False:
        pass
    elif index % cycle_size == 3:
        return [
            {"role": "user", "content": "Translate this problem into Python code.\n\n" + question + "\n\nStart by importing numpy and sympy. If you get a confident answer after running the sympy code, put your final answer within \boxed{}"}
        ]
    elif index % cycle_size == 2:
        return [
            {"role": "user", "content": "Translate the following problem into sympy.\n\n" + question + "\n\nStart by importing sympy. If you get a confident answer after running the sympy code, put your final answer within \boxed{}"}
        ]
    elif index % cycle_size == 1:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        return [
            {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
            {"role": "user", "content": question}
        ]
    else:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        return [
            {"role": "system", "content": "Please integrate natural language reasoning with Python programs to solve the problem above, and put your final answer within \\boxed{}."},
            {"role": "user", "content": question}
        ]

In [11]:
def predict_for_question(question: str) -> int:
    import os
    # if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    #     if question != "Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?":
    #         return 210
    
    # question += "\nIf the final answer is a number larger than 1 million, take modulo 1000."
    print(question)

    list_of_messages = [create_starter_messages(question, index) for index in range(16)]

    all_extracted_answers = []
    for _ in range(4):
        list_of_messages = batch_message_generate(list_of_messages)
        list_of_messages, extracted_answers = batch_message_filter(list_of_messages)
        all_extracted_answers.extend(extracted_answers)
        if not list_of_messages:
            break
        list_of_messages = batch_message_execute(list_of_messages)

    print(all_extracted_answers)
    answer = select_answer(all_extracted_answers)
    print(answer)

    print("\n\n")
    return answer

In [14]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    ids = id_['id'].to_list()  # Extract the 'id' column as a list
    questions = question['problem'].to_list()  # Extract the 'problem' column as a list
    
    answers = []
    for q in questions:
        answer = predict_for_question(q)
        answers.append(answer)
    
    result_df = pl.DataFrame({'id': ids, 'answer': answers})
    return result_df

In [ ]:
reference_df = pl.read_csv('reference.csv')
reference_df = reference_df.drop('answer')
id_df = reference_df.select('id')
question_df = reference_df.select('problem')
result_df = predict(id_df, question_df)
result_df.to_csv("submission2.csv")
# Print the result
print(result_df)

Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?


Processed prompts: 100%|█| 16/16 [00:43<00:00,  2.72s/it, est. speed input: 51.3


cecococecoceco


Processed prompts: 100%|█| 7/7 [00:43<00:00,  6.19s/it, est. speed input: 152.25


cococece


...

WARNING 10-29 17:42:52 scheduler.py:895] Input prompt (4112 tokens) is too long and exceeds limit of 4096
WARNING 10-29 17:42:52 scheduler.py:895] Input prompt (4116 tokens) is too long and exceeds limit of 4096


Processed prompts: 100%|█| 2/2 [00:00<00:00, 841.98it/s, est. speed input: 37240

c

oco
['\\infty', '1', '2', '2', '\\infty', '2', '2', '2', '1', '2', '3', '3', '2', '3']
2



Let $ABC$ be a triangle with $BC=108$, $CA=126$, and $AB=39$. Point $X$ lies on segment $AC$ such that $BX$ bisects $\angle CBA$. Let $\omega$ be the circumcircle of triangle $ABX$. Let $Y$ be a point on $\omega$ different from $X$ such that $CX=CY$. Line $XY$ meets $BC$ at $E$. The length of the segment $BE$ can be written as $\frac{m}{n}$, where $m$ and $n$ are coprime positive integers. Find $m+n$.


Processed prompts:  94%|▉| 15/16 [00:31<00:02,  2.37s/it, est. speed input: 90.9